# P4

# Nombre: Joel Quilumba

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
# from src import eliminacion_gaussiana, gauss_jordan
# las funciones se encuentran disponibles como referencia

In [2]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)
logging.info(datetime.now())

[08-14 08:52:05][INFO] 2024-08-14 08:52:05.908697


In [3]:
# ####################################################################
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``Ab``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        Ab = np.array(Ab)
    assert Ab.shape[0] == Ab.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = Ab.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # --- Sustitución hacia atrás
    solucion = np.zeros(n)

    for i in range(n - 1, -1, -1):
        solucion[i] = Ab[i, -1] / Ab[i, i]

    return solucion

In [4]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)
gauss_jordan(Ab)

UsageError: Line magic function `%autoreload` not found.


In [5]:
# ####################################################################
def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Inversión de una matriz cuadrada mediante el método de Gauss-Jordan.

    ## Parameters
    ``A``: matriz cuadrada de tamaño n x n.

    ## Return
    ``A_inv``: matriz inversa de A.
    """
    if not isinstance(A, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        A = np.array(A)
    n = A.shape[0]
    assert A.shape[0] == A.shape[1], "La matriz A debe ser cuadrada."

    Ab = np.hstack((A, np.eye(n)))

    for i in range(0, n):  
        
        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
              
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Escalonamiento: hacer el pivote 1 y eliminar los demás elementos de la columna
        Ab[i, :] = Ab[i, :] / Ab[i, i]

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    # La parte derecha de Ab ahora es la inversa de A
    A_inv = Ab[:, n:]

    return A_inv

## Ejemplos 
Compruebe su función con respecto a los siguientes ejemplos:
* Ejemplo 1

In [6]:
# La matriz A =
A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
# tiene como inversa
# A_inv =[[ 0.5, -0.5, -1.5,  1.5],
#        [-0.5,  1.5, -1.5,  0.5],
#        [-1.5, -1.5,  3.5, -1.5],
#        [ 1.5,  0.5, -1.5,  0.5]]
inv_matrix(A)

[08-14 08:52:08][INFO] 
[[ 1.  2.  3.  4.  1.  0.  0.  0.]
 [ 0.  1.  0. -1. -2.  1.  0.  0.]
 [ 0.  0. -1. -3. -3.  0.  1.  0.]
 [ 0. -1. -3. -6. -4.  0.  0.  1.]]
[08-14 08:52:08][INFO] 
[[ 1.  0.  3.  6.  5. -2.  0.  0.]
 [ 0.  1.  0. -1. -2.  1.  0.  0.]
 [ 0.  0. -1. -3. -3.  0.  1.  0.]
 [ 0.  0. -3. -7. -6.  1.  0.  1.]]
[08-14 08:52:08][INFO] 
[[ 1.  0.  0. -3. -4. -2.  3.  0.]
 [ 0.  1.  0. -1. -2.  1.  0.  0.]
 [-0. -0.  1.  3.  3. -0. -1. -0.]
 [ 0.  0.  0.  2.  3.  1. -3.  1.]]
[08-14 08:52:08][INFO] 
[[ 1.   0.   0.   0.   0.5 -0.5 -1.5  1.5]
 [ 0.   1.   0.   0.  -0.5  1.5 -1.5  0.5]
 [-0.  -0.   1.   0.  -1.5 -1.5  3.5 -1.5]
 [ 0.   0.   0.   1.   1.5  0.5 -1.5  0.5]]


array([[ 0.5, -0.5, -1.5,  1.5],
       [-0.5,  1.5, -1.5,  0.5],
       [-1.5, -1.5,  3.5, -1.5],
       [ 1.5,  0.5, -1.5,  0.5]])

* Ejemplo 2

In [7]:
# La matriz A =
A = [
    [4, 4, 5, 1],
    [3, 4, 2, 2],
    [2, 1, 4, 1],
    [3, 2, 5, 4],
]
# tiene como inversa
# A_inv =[[-34.,  31.,  52., -20.],
#         [ 19., -17., -29.,  11.],
#         [ 12., -11., -18.,   7.],
#         [  1.,  -1.,  -2.,   1.]]
inv_matrix(A)

[08-14 08:52:08][INFO] 
[[ 1.   0.5  2.   0.5  0.   0.   0.5  0. ]
 [ 0.   2.5 -4.   0.5  0.   1.  -1.5  0. ]
 [ 0.   2.  -3.  -1.   1.   0.  -2.   0. ]
 [ 0.   0.5 -1.   2.5  0.   0.  -1.5  1. ]]
[08-14 08:52:08][INFO] 
[[  1.   0.   3.  -2.   0.   0.   2.  -1.]
 [  0.   1.  -2.   5.   0.   0.  -3.   2.]
 [  0.   0.   1. -11.   1.   0.   4.  -4.]
 [  0.   0.   1. -12.   0.   1.   6.  -5.]]
[08-14 08:52:08][INFO] 
[[  1.   0.   0.  31.  -3.   0. -10.  11.]
 [  0.   1.   0. -17.   2.   0.   5.  -6.]
 [  0.   0.   1. -11.   1.   0.   4.  -4.]
 [  0.   0.   0.  -1.  -1.   1.   2.  -1.]]
[08-14 08:52:08][INFO] 
[[  1.   0.   0.   0. -34.  31.  52. -20.]
 [  0.   1.   0.   0.  19. -17. -29.  11.]
 [  0.   0.   1.   0.  12. -11. -18.   7.]
 [ -0.  -0.  -0.   1.   1.  -1.  -2.   1.]]


array([[-34.,  31.,  52., -20.],
       [ 19., -17., -29.,  11.],
       [ 12., -11., -18.,   7.],
       [  1.,  -1.,  -2.,   1.]])

# Ejercicios
* Ejercicio 1

In [8]:
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]
inv_matrix(A)

[08-14 08:52:09][INFO] 
[[  1.   0.   4.   0.   0.   1.   0.   0.]
 [  0.   0. -16.   5.   1.  -4.   0.   0.]
 [  0.   4. -11.   3.   0.  -3.   1.   0.]
 [  0.   3.  -1.   0.   0.  -1.   0.   1.]]
[08-14 08:52:09][INFO] 
[[  1.           0.           4.           0.           0.
    1.           0.           0.        ]
 [  0.           1.          -0.33333333   0.           0.
   -0.33333333   0.           0.33333333]
 [  0.           0.          -9.66666667   3.           0.
   -1.66666667   1.          -1.33333333]
 [  0.           0.         -16.           5.           1.
   -4.           0.           0.        ]]
[08-14 08:52:09][INFO] 
[[ 1.          0.          0.          1.24137931  0.          0.31034483
   0.4137931  -0.55172414]
 [ 0.          1.          0.         -0.10344828  0.         -0.27586207
  -0.03448276  0.37931034]
 [-0.         -0.          1.         -0.31034483 -0.          0.17241379
  -0.10344828  0.13793103]
 [ 0.          0.          0.          0.034482

array([[-36.,  45.,  60., -80.],
       [  3.,  -4.,  -5.,   7.],
       [  9., -11., -15.,  20.],
       [ 29., -36., -48.,  64.]])

* Ejercicio 2

In [9]:
A = [
    [0, 0, 0, 0, 0, 0, 1, -1],
    [0, 1, -1, 1, 0, -1, 0, 1],
    [-1, -1, 0, 0, 2, 1, 0, 0],
    [-1, -1, -1, 1, 2, 0, 0, 1],
    [-1, 1, 1, 0, -1, -1, 0, 2],
    [0, 1, 0, 0, -1, -1, 0, 0],
    [1, -1, -1, 1, 2, 1, 0, 2],
    [2, 0, 0, 0, 0, 1, 2, 0],
]
inv_matrix(A)

[08-14 08:52:10][INFO] 
[[ 1.  1. -0. -0. -2. -1. -0. -0. -0. -0. -1. -0. -0. -0. -0. -0.]
 [ 0.  1. -1.  1.  0. -1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0. -1.  0.  1.  0.  0. -1.  1.  0.  0.  0.  0.]
 [ 0.  2.  1.  0. -3. -2.  0.  2.  0.  0. -1.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0. -1. -1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0. -2. -1.  1.  4.  2.  0.  2.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0. -2.  0.  0.  4.  3.  2.  0.  0.  0.  2.  0.  0.  0.  0.  1.]]
[08-14 08:52:10][INFO] 
[[ 1.  0.  1. -1. -2.  0. -0. -1. -0. -1. -1. -0. -0. -0. -0. -0.]
 [ 0.  1. -1.  1.  0. -1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0. -1.  0.  1.  0.  0. -1.  1.  0.  0.  0.  0.]
 [ 0.  0.  3. -2. -3.  0.  0.  0.  0. -2. -1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1. -1. -1.  0.  0. -1.  0. -1.  0.  0.  0.  1.  0.  0.]
 [ 0.  0. -3.

array([[ 2., -1.,  0., -1.,  0.,  2.,  2., -1.],
       [ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 6., -1.,  0., -3.,  1.,  1.,  4., -3.],
       [ 6.,  1., -1., -3.,  1., -3.,  3., -3.],
       [ 2., -1.,  1., -1.,  0.,  3.,  2., -1.],
       [-2.,  2.,  0.,  0.,  0., -4., -2.,  1.],
       [-1.,  0.,  0.,  1.,  0.,  0., -1.,  1.],
       [-2.,  0.,  0.,  1.,  0.,  0., -1.,  1.]])

* Ejercicio 3

In [26]:
A = [
    [1, 0, 0, 0, -1, 0, 0, -1, 1, -1],
    [1, 1, 0, -1, -1, 1, 0, 0, 1, -1],
    [-1, 0, -1, 0, 0, 0, -1, 1, 0, 0],
    [0, 0, -1, 0, -1, -1, 1, 0, 1, 0],
    [-1, 0, 0, -1, 1, 1, 1, 1, 0, -1],
    [1, 0, 0, 1, -1, -1, -1, 1, -1, 0],
    [1, 1, 1, 0, 1, 0, -1, -1, -1, 1],
    [1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, -1, 0, 1, 1, 1, -1],
]
inv_matrix(A)

[08-14 08:42:50][INFO] 
[[ 1.  0.  0.  0. -1.  0.  0. -1.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  1.  0. -1.  0.  1.  0.  1.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0. -1.  0. -1.  0. -1.  0.  1. -1.  1.  0.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0. -1.  0. -1. -1.  1.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0. -1.  0.  1.  1.  0.  1. -2.  1.  0.  0.  0.  1.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  0. -1. -1.  2. -2.  1. -1.  0.  0.  0.  0.  1.  0.  0.
   0.  0.]
 [ 0.  1.  1.  0.  2.  0. -1.  0. -2.  2. -1.  0.  0.  0.  0.  0.  1.  0.
   0.  0.]
 [ 0.  1.  1.  1.  1.  0.  1.  2. -1.  1. -1.  0.  0.  0.  0.  0.  0.  1.
   0.  0.]
 [ 0.  1.  1.  1.  2.  0. -1.  0. -1.  1. -1.  0.  0.  0.  0.  0.  0.  0.
   1.  0.]
 [ 0.  0. -1. -1. -1.  0.  1.  1.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  1.]]
[08-14 08:42:50][INFO] 
[[ 1.  0.  0.  0. -1.  0.  0. -1.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  1.  0. -1

array([[ 14.,  -8.,   9.,  -4.,   0.,  -4.,   9.,   7.,  -8.,   3.],
       [ -2.,   2.,  -1.,   2.,   1.,   1.,  -1.,  -1.,   1.,  -2.],
       [-27.,  14., -18.,   5.,  -2.,   7., -17., -13.,  16.,  -2.],
       [ 12.,  -6.,   8.,  -2.,   1.,  -3.,   7.,   6.,  -7.,   0.],
       [  6.,  -4.,   4.,  -2.,   0.,  -2.,   4.,   3.,  -3.,   2.],
       [ 18.,  -9.,  12.,  -4.,   1.,  -5.,  11.,   9., -11.,   1.],
       [  8.,  -4.,   5.,  -1.,   1.,  -2.,   5.,   4.,  -5.,   0.],
       [ -5.,   2.,  -3.,   0.,  -1.,   1.,  -3.,  -2.,   3.,   1.],
       [-11.,   5.,  -7.,   1.,  -2.,   2.,  -7.,  -5.,   7.,   1.],
       [  1.,  -1.,   1.,  -1.,  -1.,  -1.,   1.,   1.,  -1.,   1.]])